# MongoDB Aggregation Pipeline In Python

Learn about the various stages and configurations you can create to configure an Aggregation Pipeline

Stages: [match](#match), [project](#project), [unset](#unset), [limit](#limit), [skip](#skip), [sort](#sort), [count](#count), [sortByCount](#sortByCount), [unwind](#unwind), [group](#group), [addFields](#addFields), [sample](#sample), [lookup](#lookup), [unionWith](#unionWith), [out](#out), [merge](#merge)

Operators: [size](#size-(operator)), [in](#in-(operator)), [arrayElemAt](#arrayElemAt-(operator)), [first](#first-(operator)), [count](#count-(accumulator-operator)), [sum](#sum-(accumulator-operator)), [first, last](#first,-last-(accumulator-operators)), [push](#push-(accumulator-operator)), [addToSet](#addToSet-(accumulator-operator)), [regexMatch](#regexMatch-(operator)), [cond](#cond-(operator)), [Date](#Date-Operators), [expr](#expr-(operator)), [ifNull](#ifNull-(operator)), [type](#type-(operator)), [switch](#switch-(operator))

In [1]:
from pymongo import MongoClient

In [2]:
mongodb_uri = "mongodb://localhost:27017/"
db_name = "aggregation_test"

In [3]:
client = MongoClient(mongodb_uri)
db = client[db_name]

### Helper Function

In [4]:
def print_cursor(cursor):
    for document in cursor:
        print(document, end="\n\n")

### Inserting Some Sample Data

In [7]:
import insert_aggregation_sample_data as iasd
iasd.insert_data(mongodb_uri, db_name)

ModuleNotFoundError: No module named 'insert_aggregation_sample_data'

### match

In [5]:
print_cursor(
db.products.aggregate([
    {"$match": {"name": "Pens"}}
]))

{'_id': ObjectId('67e19b0e891479c315f80784'), 'name': 'Pens', 'seller_id': ObjectId('67e19b0e891479c315f8077d'), 'tags': ['Office', 'School']}



In [22]:
print_cursor(
    db.products.aggregate([
        {"$match": {"tags": {"$in": ["Beauty"]}}}
    ])
)

# print_cursor(
#     db.products.aggregate([
#         {"$match": {
#             "tags": "Home"
#         }}
#     ])
# )

{'_id': ObjectId('67e19b0e891479c315f80783'), 'name': 'Moisturizer', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80785'), 'name': 'Face Cleanser', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80786'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('67e19b0e891479c315f8077f'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80787'), 'name': 'Eyeliner', 'seller_id': ObjectId('67e19b0e891479c315f80780'), 'tags': ['Beauty']}



In [20]:
print_cursor(db.products.aggregate([
    {
        "$match": {
            "$or": [
                {"tags": "Beauty"},
                {"tags": "Home"}
            ]
        }
    }
]))

{'_id': ObjectId('67e19b0e891479c315f80782'), 'name': 'Mug', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('67e19b0e891479c315f80783'), 'name': 'Moisturizer', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80785'), 'name': 'Face Cleanser', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80786'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('67e19b0e891479c315f8077f'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80787'), 'name': 'Eyeliner', 'seller_id': ObjectId('67e19b0e891479c315f80780'), 'tags': ['Beauty']}



### project

In [11]:
cursor = db.products.aggregate([
    {"$project": {"_id": 0, "project_name": "$name", "tags": 1}}
])

In [24]:
print_cursor(db.products.aggregate([
    {"$match": {"name": "Pens"}},
    {"$project": {"tags": 1, "product_name": "$name", "_id": 0}}
]))

{'tags': ['Office', 'School'], 'product_name': 'Pens'}



### unset

In [23]:
print_cursor(db.products.aggregate([
    {"$match": {"name": "Pens"}},
    {"$unset": ["_id"]}
]))

{'name': 'Pens', 'seller_id': ObjectId('67e19b0e891479c315f8077d'), 'tags': ['Office', 'School']}



### limit

In [16]:
cur = db.products.aggregate([
    {"$limit": 3}
])
print_cursor(cur)

{'_id': ObjectId('67e19b0e891479c315f80782'), 'name': 'Mug', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('67e19b0e891479c315f80783'), 'name': 'Moisturizer', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80784'), 'name': 'Pens', 'seller_id': ObjectId('67e19b0e891479c315f8077d'), 'tags': ['Office', 'School']}



### skip

In [17]:
cur = db.products.aggregate([
    {"$skip": 1},
    {"$limit": 3}
])
print_cursor(cur)

{'_id': ObjectId('67e19b0e891479c315f80783'), 'name': 'Moisturizer', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80784'), 'name': 'Pens', 'seller_id': ObjectId('67e19b0e891479c315f8077d'), 'tags': ['Office', 'School']}

{'_id': ObjectId('67e19b0e891479c315f80785'), 'name': 'Face Cleanser', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}



### sort

In [18]:
cur = db.products.aggregate([
    {"$sort": {"name": 1}}
])
print_cursor(cur)

{'_id': ObjectId('67e19b0e891479c315f80786'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('67e19b0e891479c315f8077f'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80787'), 'name': 'Eyeliner', 'seller_id': ObjectId('67e19b0e891479c315f80780'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80785'), 'name': 'Face Cleanser', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80783'), 'name': 'Moisturizer', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80782'), 'name': 'Mug', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('67e19b0e891479c315f80784'), 'name': 'Pens', 'seller_id': ObjectId('67e19b0e891479c315f8077d'), 'tags': ['Office', 'School']}



### count

In [19]:
cur = db.products.aggregate([
    {"$match": {"tags": "Beauty"}},
    {"$count": "beauty_products_count"}
])
print_cursor(cur)

{'beauty_products_count': 4}



### sortByCount

In [20]:
cur = db.products.aggregate([
    {"$sortByCount": "$tags"}
])
print_cursor(cur)

{'_id': ['Beauty'], 'count': 4}

{'_id': ['Home', 'Kitchen'], 'count': 1}

{'_id': ['Office', 'School'], 'count': 1}



### size (operator)

In [21]:
cur = db.products.aggregate([
    {"$project": {"_id": 0, "nums_of_tags": {"$size": "$tags"}, "tags": 1}}
])
print_cursor(cur)

{'tags': ['Home', 'Kitchen'], 'nums_of_tags': 2}

{'tags': ['Beauty'], 'nums_of_tags': 1}

{'tags': ['Office', 'School'], 'nums_of_tags': 2}

{'tags': ['Beauty'], 'nums_of_tags': 1}

{'tags': ['Beauty'], 'nums_of_tags': 1}

{'tags': ['Beauty'], 'nums_of_tags': 1}



### in (operator)

In [22]:
cur = db.products.aggregate([
    {
        "$project": {
            "_id": 0,
            "is_beauty_product": {
                "$in": ["beauty", "$tags"]
            },
            "tags": 1  # Move "tags" outside of "$in"
        }
    }
])


In [23]:
print_cursor(cur)

{'tags': ['Home', 'Kitchen'], 'is_beauty_product': False}

{'tags': ['Beauty'], 'is_beauty_product': False}

{'tags': ['Office', 'School'], 'is_beauty_product': False}

{'tags': ['Beauty'], 'is_beauty_product': False}

{'tags': ['Beauty'], 'is_beauty_product': False}

{'tags': ['Beauty'], 'is_beauty_product': False}



### arrayElemAt (operator)

In [24]:
cur = db.products.aggregate([
    {
        "$project": {"_id": 0, "name": 1, "first_tag": {"$arrayElemAt": ["$tags", 0]}, "tags": "$tags"}
    }
])
print_cursor(cur)

{'name': 'Mug', 'first_tag': 'Home', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Pens', 'first_tag': 'Office', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'first_tag': 'Beauty', 'tags': ['Beauty']}



### first (operator)

In [25]:
cur = db.products.aggregate([
    {
        "$project": {"_id": 0, "name": 1, "first_tag": {"$first": "$tags"}, "tags": "$tags"}
    }
])
print_cursor(cur)

{'name': 'Mug', 'first_tag': 'Home', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Pens', 'first_tag': 'Office', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'first_tag': 'Beauty', 'tags': ['Beauty']}



### unwind

In [30]:
print_cursor(db.products.aggregate([
    {"$unwind": "$tags"},
    {"$unset": ["_id", "seller_id"]}
]))

{'name': 'Mug', 'tags': 'Home'}

{'name': 'Mug', 'tags': 'Kitchen'}

{'name': 'Moisturizer', 'tags': 'Beauty'}

{'name': 'Pens', 'tags': 'Office'}

{'name': 'Pens', 'tags': 'School'}

{'name': 'Face Cleanser', 'tags': 'Beauty'}

{'name': 'Concealer Makeup', 'tags': 'Beauty'}

{'name': 'Eyeliner', 'tags': 'Beauty'}



In [45]:
print_cursor(
    db.orders.aggregate([
        {"$project": {"items": 1, "_id": 0}}
    ])
)

{'items': [{'product_id': ObjectId('67e19b0e891479c315f80783'), 'quantity': 1}, {'product_id': ObjectId('67e19b0e891479c315f80785'), 'quantity': 1}]}

{'items': [{'product_id': ObjectId('67e19b0e891479c315f80786'), 'quantity': 1}, {'product_id': ObjectId('67e19b0e891479c315f80787'), 'quantity': 1}]}

{'items': [{'product_id': ObjectId('67e19b0e891479c315f80784'), 'quantity': 5}, {'product_id': ObjectId('67e19b0e891479c315f80782'), 'quantity': 1}]}

{'items': [{'product_id': ObjectId('67e19b0e891479c315f80783'), 'quantity': 2}, {'product_id': ObjectId('67e19b0e891479c315f80787'), 'quantity': 1}]}

{'items': [{'product_id': ObjectId('67e19b0e891479c315f80783'), 'quantity': 1}]}

{'items': [{'product_id': ObjectId('67e19b0e891479c315f80785'), 'quantity': 1}]}



In [52]:
cursor = db.products.aggregate([
        {"$match": {"tags": {"$size": 2}}},
        {"$unwind":  {"path": "$tags", "includeArrayIndex": "some_index"}},
        {"$unset": ["_id", "seller_id"]}
    ])
print_cursor(cursor)

{'name': 'Mug', 'tags': 'Home', 'some_index': 0}

{'name': 'Mug', 'tags': 'Kitchen', 'some_index': 1}

{'name': 'Pens', 'tags': 'Office', 'some_index': 0}

{'name': 'Pens', 'tags': 'School', 'some_index': 1}



### group

In [62]:
print_cursor(db.products.aggregate([
    {"$group": {"_id": "$tags", "numOfCount": {"$count": {}}}},
]))


{'_id': ['Beauty'], 'numOfCount': 4}

{'_id': ['Office', 'School'], 'numOfCount': 1}

{'_id': ['Home', 'Kitchen'], 'numOfCount': 1}



In [72]:
print_cursor(
    db.orders.aggregate([
        {"$match": {}}
    ])
)

{'_id': ObjectId('67e19b0e891479c315f80788'), 'items': [{'product_id': ObjectId('67e19b0e891479c315f80783'), 'quantity': 1}, {'product_id': ObjectId('67e19b0e891479c315f80785'), 'quantity': 1}]}

{'_id': ObjectId('67e19b0e891479c315f80789'), 'items': [{'product_id': ObjectId('67e19b0e891479c315f80786'), 'quantity': 1}, {'product_id': ObjectId('67e19b0e891479c315f80787'), 'quantity': 1}]}

{'_id': ObjectId('67e19b0e891479c315f8078a'), 'items': [{'product_id': ObjectId('67e19b0e891479c315f80784'), 'quantity': 5}, {'product_id': ObjectId('67e19b0e891479c315f80782'), 'quantity': 1}]}

{'_id': ObjectId('67e19b0e891479c315f8078b'), 'items': [{'product_id': ObjectId('67e19b0e891479c315f80783'), 'quantity': 2}, {'product_id': ObjectId('67e19b0e891479c315f80787'), 'quantity': 1}]}

{'_id': ObjectId('67e19b0e891479c315f8078c'), 'items': [{'product_id': ObjectId('67e19b0e891479c315f80783'), 'quantity': 1}]}

{'_id': ObjectId('67e19b0e891479c315f8078d'), 'items': [{'product_id': ObjectId('67e19b0e

In [70]:
print_cursor(
    db.orders.aggregate([
        {"$unwind": "$items"},
        {"$group": {"_id": "$items.product_id", "total_quantity": {"$sum": "$items.quantity"}}}
    ])
)

{'_id': ObjectId('67e19b0e891479c315f80786'), 'total_quantity': 1}

{'_id': ObjectId('67e19b0e891479c315f80785'), 'total_quantity': 2}

{'_id': ObjectId('67e19b0e891479c315f80784'), 'total_quantity': 5}

{'_id': ObjectId('67e19b0e891479c315f80782'), 'total_quantity': 1}

{'_id': ObjectId('67e19b0e891479c315f80787'), 'total_quantity': 2}

{'_id': ObjectId('67e19b0e891479c315f80783'), 'total_quantity': 4}



### count (accumulator operator)

### sum (accumulator operator)

In [79]:
print_cursor(
    db.products.aggregate([
        {"$match": {}}
    ])
)

{'_id': ObjectId('67e19b0e891479c315f80782'), 'name': 'Mug', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('67e19b0e891479c315f80783'), 'name': 'Moisturizer', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80784'), 'name': 'Pens', 'seller_id': ObjectId('67e19b0e891479c315f8077d'), 'tags': ['Office', 'School']}

{'_id': ObjectId('67e19b0e891479c315f80785'), 'name': 'Face Cleanser', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80786'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('67e19b0e891479c315f8077f'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80787'), 'name': 'Eyeliner', 'seller_id': ObjectId('67e19b0e891479c315f80780'), 'tags': ['Beauty']}



### first, last (accumulator operators)

In [83]:
print_cursor(
    db.products.aggregate([
        {"$group": {"_id": "$tags", "num_entries": { "$count": {} }, "first": {"$first": "$name"}, "last": {"$last": "$name"}}}
    ])
)

{'_id': ['Home', 'Kitchen'], 'num_entries': 1, 'first': 'Mug', 'last': 'Mug'}

{'_id': ['Beauty'], 'num_entries': 4, 'first': 'Moisturizer', 'last': 'Eyeliner'}

{'_id': ['Office', 'School'], 'num_entries': 1, 'first': 'Pens', 'last': 'Pens'}



### push (accumulator operator)

In [84]:
print_cursor(db.products.aggregate([
    {"$group": {"_id": "$tags", "products": {"$push": "$name"}}}
]))

{'_id': ['Home', 'Kitchen'], 'products': ['Mug']}

{'_id': ['Beauty'], 'products': ['Moisturizer', 'Face Cleanser', 'Concealer Makeup', 'Eyeliner']}

{'_id': ['Office', 'School'], 'products': ['Pens']}



### addToSet (accumulator operator)

In [85]:
print_cursor(db.products.aggregate([
    {"$group": {"_id": "$tags", "products": {"$addToSet": "$name"}}}
]))

{'_id': ['Home', 'Kitchen'], 'products': ['Mug']}

{'_id': ['Beauty'], 'products': ['Eyeliner', 'Concealer Makeup', 'Moisturizer', 'Face Cleanser']}

{'_id': ['Office', 'School'], 'products': ['Pens']}



### $$ROOT (system variable)

In [87]:
print_cursor(db.products.aggregate([
    {"$group": {"_id": "$tags", "products": {"$addToSet": "$$ROOT"}}}
]))

{'_id': ['Home', 'Kitchen'], 'products': [{'_id': ObjectId('67e19b0e891479c315f80782'), 'name': 'Mug', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Home', 'Kitchen']}]}

{'_id': ['Beauty'], 'products': [{'_id': ObjectId('67e19b0e891479c315f80785'), 'name': 'Face Cleanser', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}, {'_id': ObjectId('67e19b0e891479c315f80786'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('67e19b0e891479c315f8077f'), 'tags': ['Beauty']}, {'_id': ObjectId('67e19b0e891479c315f80783'), 'name': 'Moisturizer', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}, {'_id': ObjectId('67e19b0e891479c315f80787'), 'name': 'Eyeliner', 'seller_id': ObjectId('67e19b0e891479c315f80780'), 'tags': ['Beauty']}]}

{'_id': ['Office', 'School'], 'products': [{'_id': ObjectId('67e19b0e891479c315f80784'), 'name': 'Pens', 'seller_id': ObjectId('67e19b0e891479c315f8077d'), 'tags': ['Office', 'School']}]}



### addFields

In [38]:
print_cursor(db.products.aggregate([
    {"$match": {"name": "Pens"}},
    {"$addFields": {"count_tags": {"$size": "$tags"}}},
    {"$unset": ["_id", "seller_id"]}
]))

{'name': 'Pens', 'tags': ['Office', 'School'], 'count_tags': 2}



### sample

In [44]:
print_cursor(db.products.aggregate([
    {"$sample": {"size": 3}},
    {"$unset": ["_id", "seller_id"]}
]))

{'name': 'Eyeliner', 'tags': ['Beauty']}

{'name': 'Moisturizer', 'tags': ['Beauty']}

{'name': 'Face Cleanser', 'tags': ['Beauty']}



### lookup

In [45]:
print_cursor(
    db.products.aggregate([
        {"$lookup": {
            "from": "users",
            "localField": "seller_id",
            "foreignField": "_id",
            "as": "seller_join"
        }}
    ])
)

{'_id': ObjectId('67e19b0e891479c315f80782'), 'name': 'Mug', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Home', 'Kitchen'], 'seller_join': [{'_id': ObjectId('67e19b0e891479c315f8077c'), 'name': 'Sarah'}]}

{'_id': ObjectId('67e19b0e891479c315f80783'), 'name': 'Moisturizer', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty'], 'seller_join': [{'_id': ObjectId('67e19b0e891479c315f8077c'), 'name': 'Sarah'}]}

{'_id': ObjectId('67e19b0e891479c315f80784'), 'name': 'Pens', 'seller_id': ObjectId('67e19b0e891479c315f8077d'), 'tags': ['Office', 'School'], 'seller_join': [{'_id': ObjectId('67e19b0e891479c315f8077d'), 'name': 'Bob'}]}

{'_id': ObjectId('67e19b0e891479c315f80785'), 'name': 'Face Cleanser', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty'], 'seller_join': [{'_id': ObjectId('67e19b0e891479c315f8077c'), 'name': 'Sarah'}]}

{'_id': ObjectId('67e19b0e891479c315f80786'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('67e19b0e891

In [50]:
print_cursor(
    db.products.aggregate([
        {"$lookup": {
            "from": "users",
            "localField": "seller_id",
            "foreignField": "_id",
            "as": "seller_join"
        }},
        {"$project": {"_id": 0, "product_name": "$name", "seller_name": {"$first": "$seller_join.name"}}}
    ])
)

{'product_name': 'Mug', 'seller_name': 'Sarah'}

{'product_name': 'Moisturizer', 'seller_name': 'Sarah'}

{'product_name': 'Pens', 'seller_name': 'Bob'}

{'product_name': 'Face Cleanser', 'seller_name': 'Sarah'}

{'product_name': 'Concealer Makeup', 'seller_name': 'Lisa'}

{'product_name': 'Eyeliner', 'seller_name': 'Jessica'}



In [12]:
print_cursor(
    db.users.aggregate([
        {"$lookup": {
            "from": "products",
            "localField": "_id",
            "foreignField": "seller_id",
            "as": "products"
        }},
        {"$addFields": {"num_products": {"$size": "$products"}}},
        {"$match": {"num_products": {"$gte": 1}}},
        {"$project": {"_id": 0, "product_name": "$name", "products": "$products.name"}}
    ])
)

{'product_name': 'Sarah', 'products': ['Mug', 'Moisturizer', 'Face Cleanser']}

{'product_name': 'Bob', 'products': ['Pens']}

{'product_name': 'Lisa', 'products': ['Concealer Makeup']}

{'product_name': 'Jessica', 'products': ['Eyeliner']}



In [23]:
print_cursor(
    db.products.aggregate([
        {"$group": {"_id": "$seller_id", "product_name": {"$push": "$name"}}},
        {"$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "_id",
            "as": "sellers"
        }},
        {"$project": {
            "_id": 0, 
            "seller_name": {"$first": "$sellers.name"}, 
            "products": "$product_name"
        }}
    ])
)


{'seller_name': 'Sarah', 'products': ['Mug', 'Moisturizer', 'Face Cleanser']}

{'seller_name': 'Lisa', 'products': ['Concealer Makeup']}

{'seller_name': 'Jessica', 'products': ['Eyeliner']}

{'seller_name': 'Bob', 'products': ['Pens']}



### unionWith

In [28]:
print_cursor(
    db.products.aggregate([
        {"$unionWith": "users"}
    ])
)


{'_id': ObjectId('67e19b0e891479c315f80782'), 'name': 'Mug', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('67e19b0e891479c315f80783'), 'name': 'Moisturizer', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80784'), 'name': 'Pens', 'seller_id': ObjectId('67e19b0e891479c315f8077d'), 'tags': ['Office', 'School']}

{'_id': ObjectId('67e19b0e891479c315f80785'), 'name': 'Face Cleanser', 'seller_id': ObjectId('67e19b0e891479c315f8077c'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80786'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('67e19b0e891479c315f8077f'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f80787'), 'name': 'Eyeliner', 'seller_id': ObjectId('67e19b0e891479c315f80780'), 'tags': ['Beauty']}

{'_id': ObjectId('67e19b0e891479c315f8077c'), 'name': 'Sarah'}

{'_id': ObjectId('67e19b0e891479c315f8077d'), 'name': 'Bob'}

{'_id': ObjectId('67

### regexMatch (operator)

In [ ]:
user_search = "is"
print_cursor(
    db.products.aggregate([
        {"$unionWith": "users"},
        {"$match": {"name": {"$regex": user_search, "$options": "i"}}}
    ])
)


### out
*Note: You can potentially overwrite all your data in a collection with this stage, use with caution*

In [30]:
print_cursor(
    db.products.aggregate([
        {"$match": {"tags": "Beauty"}},
        {"$out": {"db": "aggregate_test", "coll": "beauty_products"}}
    ])
)

### merge
*Note: You can potentially overwrite data within a collection with this stage, use with caution*

In [39]:
print_cursor(
    db.products.aggregate([
        {"$match": {"tags": "Kitchen"}},
        {
            "$merge": {
                "into": {"db": "aggregation_test", "coll": "beauty_products"},
                "on": "_id",
                "whenMatched": "replace",
                "whenNotMatched": "insert"
            }

         }
    ])
)

### cond (operator)

In [ ]:
def dosomething(param1, param2):
    return f"This is doing something and returning the response {param1} and {param2}"

print_cursor(
    db.products.aggregate([
        {"$project": {"_id": 0, "name": 1, "is_bob": {
            "$cond": {"if": {"$eq": ["$name", "Bob"]}, "then": True, "else": dosomething(2, 4)}
        }}}
    ])
)

{'name': 'Mug', 'is_bob': 'This is doing something and returning the response 2 and 4'}

{'name': 'Moisturizer', 'is_bob': 'This is doing something and returning the response 2 and 4'}

{'name': 'Pens', 'is_bob': 'This is doing something and returning the response 2 and 4'}

{'name': 'Face Cleanser', 'is_bob': 'This is doing something and returning the response 2 and 4'}

{'name': 'Concealer Makeup', 'is_bob': 'This is doing something and returning the response 2 and 4'}

{'name': 'Eyeliner', 'is_bob': 'This is doing something and returning the response 2 and 4'}



In [14]:
print_cursor(
    db.products.aggregate([
        {"$project": {"_id": 0, "name": 1, "is_beauty_products": {
            "$cond": {"if": {"$in": ["Beauty", "$tags"]}, "then": True, "else": False}
        }}}
    ])
)

{'name': 'Mug', 'is_beauty_products': False}

{'name': 'Moisturizer', 'is_beauty_products': True}

{'name': 'Pens', 'is_beauty_products': False}

{'name': 'Face Cleanser', 'is_beauty_products': True}

{'name': 'Concealer Makeup', 'is_beauty_products': True}

{'name': 'Eyeliner', 'is_beauty_products': True}



### $$NOW (system variable)

In [18]:
print_cursor(
    db.products.aggregate([
        {"$match": {"name": "Mug"}},
        {"$project": {"_id": 0, "date_joined": "$$NOW"}}
    ])
)

{'date_joined': datetime.datetime(2025, 3, 30, 10, 19, 3, 913000)}



In [20]:
print_cursor(
    db.products.aggregate([
        {"$match": {"name": "Mug"}},
        {"$addFields": {"date_joined": "$$NOW"}},
        {"$project": {"_id": 0, "name": 1, "premium_exp_date": {
            "$dateAdd": {"startDate": "$date_joined", "unit": "day", "amount": 7}
        }}}
    ])
)

{'name': 'Mug', 'premium_exp_date': datetime.datetime(2025, 4, 6, 10, 22, 37, 489000)}



In [ ]:
print_cursor(
    db.products.aggregate([
        # Match products with the name "Mug"
        {"$match": {"name": "Mug"}},
        # Add the current date/time as "date_joined"
        {"$addFields": {"date_joined": {"$toDate": "$$NOW"}}},
        # Project the required fields and calculate "premium_exp_date"
        {
            "$project": {
                "_id": 0, "name": 1, "date_joined": 1, "premium_exp_date": {
                    "$dateAdd": {
                        "startDate": "$date_joined", "unit": "week", "amount": 2
                    }
                }
            }
        },
        # Add "premium_days_left" by calculating the difference in days
        {
            "$addFields": {
                "premium_days_left": {
                    "$dateDiff": {
                        "startDate": "$date_joined", "endDate": "$premium_exp_date", "unit": "day"
                    }
                }
            }
        }
    ])
)


{'name': 'Mug', 'date_joined': datetime.datetime(2025, 3, 30, 15, 23, 11, 432000), 'premium_exp_date': datetime.datetime(2025, 4, 13, 15, 23, 11, 432000), 'premium_days_left': 14}



### Date Operators

In [23]:
print_cursor(
    db.products.aggregate([
        {"$match": {"name": "Mug"}},
        {"$addFields": {"date_joined": "$$NOW"}},
        {"$project": {"_id": 0, "name": 1, "date_joined_parts": {
            "$dateToParts": {"date": "$date_joined"}
        }}} 
    ])
)

{'name': 'Mug', 'date_joined_parts': {'year': 2025, 'month': 3, 'day': 30, 'hour': 15, 'minute': 25, 'second': 17, 'millisecond': 474}}



### expr (operator)

In [ ]:
print_cursor(
    db.products.aggregate([
        # Match products with the name "Mug"
        {"$match": {"name": "Mug"}},
        # Add the current date/time as "date_joined"
        {"$addFields": {"date_joined": {"$toDate": "$$NOW"}}},
        # Project the required fields and calculate "premium_exp_date"
        {
            "$project": {
                "_id": 0, "name": 1, "date_joined": 1, "premium_exp_date": {
                    "$dateAdd": {
                        "startDate": "$date_joined", "unit": "week", "amount": 7
                    }
                }
            }
        },
        # Add "premium_days_left" by calculating the difference in days
        {
            "$addFields": {
                "premium_days_left": {
                    "$dateDiff": {"startDate": "$date_joined", "endDate": "$premium_exp_date", "unit": "day"}
                }
            }
        },
        {"$match": {"premium_days_left": {"$gt": 10}}}
    ])
)

{'name': 'Mug', 'date_joined': datetime.datetime(2025, 3, 30, 15, 28, 40, 186000), 'premium_exp_date': datetime.datetime(2025, 5, 18, 15, 28, 40, 186000), 'premium_days_left': 49}



In [ ]:
print_cursor(
    db.products.aggregate([
        {"$match": {"name": "Mug"}},
        {"$addFields": {"date_joined": {"$toDate": "$$NOW"}}},
        {
            "$project": {
                "_id": 0, "name": 1, "date_joined": 1, "premium_exp_date": {
                    "$dateAdd": {"startDate": "$date_joined", "unit": "week", "amount": 7}
                }
            }
        },
        {"$match": {"$expr": {"$gt": [{"$dateDiff": {"startDate": "$date_joined", "endDate": "$premium_exp_date", "unit": "day"}}, 10]}}}
    ])
)

{'name': 'Mug', 'date_joined': datetime.datetime(2025, 3, 30, 15, 30, 6, 633000), 'premium_exp_date': datetime.datetime(2025, 5, 18, 15, 30, 6, 633000)}



### ifNull (operator)

In [28]:
print_cursor(
    db.products.aggregate([
        {"$project": {"_id": 0, "name": 1, "premium_exp_page": 1}}
    ])
)

{'name': 'Mug'}

{'name': 'Moisturizer'}

{'name': 'Pens'}

{'name': 'Face Cleanser'}

{'name': 'Concealer Makeup'}

{'name': 'Eyeliner'}



In [29]:
print_cursor(
    db.products.aggregate([
        {"$project": {"_id": 0, "name": 1, "has_premium_exp_page": {
            "$ifNull": ["$premium_exp_page", False]
        }}}
    ])
)

{'name': 'Mug', 'has_premium_exp_page': False}

{'name': 'Moisturizer', 'has_premium_exp_page': False}

{'name': 'Pens', 'has_premium_exp_page': False}

{'name': 'Face Cleanser', 'has_premium_exp_page': False}

{'name': 'Concealer Makeup', 'has_premium_exp_page': False}

{'name': 'Eyeliner', 'has_premium_exp_page': False}



### type (operator)

In [30]:
print_cursor(db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "name_type": {"$type": "$name"}}}
]))

{'name': 'Mug', 'name_type': 'string'}

{'name': 'Moisturizer', 'name_type': 'string'}

{'name': 'Pens', 'name_type': 'string'}

{'name': 'Face Cleanser', 'name_type': 'string'}

{'name': 'Concealer Makeup', 'name_type': 'string'}

{'name': 'Eyeliner', 'name_type': 'string'}



### switch (operator)

In [35]:
print_cursor(db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "user_account_error": {
        "$switch": {
            "branches": [
                {"case": {"$eq": [{"$type": "$name"}, "missing"]}, "then": "Missing Name"},
                {"case": {"$eq": [{"$type": "$premium_exp_date"}, "missing"]}, "then": "Missing Premium Exp Date"}, 
                {"case": {"$lt": ["premium_exp_date", "$$NOW"]}, "then": "Date Expired"}
            ]
        }
    }}}
]))

{'name': 'Mug', 'user_account_error': 'Missing Premium Exp Date'}

{'name': 'Moisturizer', 'user_account_error': 'Missing Premium Exp Date'}

{'name': 'Pens', 'user_account_error': 'Missing Premium Exp Date'}

{'name': 'Face Cleanser', 'user_account_error': 'Missing Premium Exp Date'}

{'name': 'Concealer Makeup', 'user_account_error': 'Missing Premium Exp Date'}

{'name': 'Eyeliner', 'user_account_error': 'Missing Premium Exp Date'}

